# Partition SBM Baseline Code

In [2]:
# License

#
# Copyright 2017 MIT Lincoln Laboratory, Massachusetts Institute of Technology
#
# Licensed under the Apache License, Version 2.0 (the "License"); you may not use these files except in compliance with
# the License.
#
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on
# an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the
# specific language governing permissions and limitations under the License.
#

"""
Based on the Python script by

Authors: Steven Smith, Edward Kao
Date: 9 January 2017
Installation: Python 2.7

Description: This Python script performs the baseline graph partition algorithm based on the degree-corrected stochastic block model.

References:
Peixoto, Tiago P. "Entropy of stochastic blockmodel ensembles." Physical Review E 85, no. 5 (2012): 056122.
Peixoto, Tiago P. "Parsimonious module inference in large networks." Physical review letters 110, no. 14 (2013): 148701.
Karrer, Brian, and Mark EJ Newman. "Stochastic blockmodels and community structure in networks." Physical Review E 83, no. 1 (2011): 016107.
"""

"Based on the Python script by\n\nAuthors: Steven Smith, Edward Kao\nDate: 9 January 2017\nInstallation: Python 2.7\n\nDescription: This Python script performs the baseline graph partition algorithm based on the degree-corrected stochastic block model.\n\nReferences:\nPeixoto, Tiago P. \"Entropy of stochastic blockmodel ensembles.\" Physical Review E 85, no. 5 (2012): 056122.\nPeixoto, Tiago P. \"Parsimonious module inference in large networks.\" Physical review letters 110, no. 14 (2013): 148701.\nKarrer, Brian, and Mark EJ Newman. \"Stochastic blockmodels and community structure in networks.\" Physical Review E 83, no. 1 (2011): 016107.\n"

In [62]:
using LightGraphs
import StatsBase: pweights, sample
using Distributions

INFO: Recompiling stale cache file /Users/rohitvarkey/.julia/lib/v0.6/Distributions.ji for module Distributions.


### Loading the Graph

We load the graph into a LightGraphs graph. We do this by reading the edges to be added and adding them onto the graph.

In [11]:
const INPUT_PATH = "data/streaming/emergingEdges/"

"data/streaming/emergingEdges/"

In [34]:
"""
Load graph given the base filename and the number of the streaming peice to be added onto a given graph.
"""
function load_graph!(g::DiGraph, filename::String, streaming_num::Int64=1)
    info("Loading $(filename)_$(streaming_num).tsv")
    edgePieces = readdlm("$(filename)_$(streaming_num).tsv")
    for i = 1:size(edgePieces, 1)
        # TODO: Add weights also
        success = add_edge!(g, edgePieces[i, 1], edgePieces[i, 2])
        if success == false
            throw("Error adding edges.")
        end
    end
end

load_graph!

### Initializing the edge counts

In [55]:
"""
Initializes the edge count matrix M between the blocks. 
Calculates the new out, in and total degrees for the updated edge count matrix.
Returns a tuple of M, d_out, d_in, d
"""
function initalize_edge_counts(g::DiGraph, B::Int64, b::Vector{Int64})
    M = zeros(Int64, B, B) # create a zero matrix of B x B 
    for v in 1:nv(g)
        for n in out_neighbors(g, v)
            # Increment count by 1
            # NOTE: Column major instead of row major
            info("Incrementing M at ($(b[n]), $(b[v]) )")
            M[b[n], b[v]] += 1
        end
    end
    # Sum across rows to get the outdegrees for each block
    d_out = reshape(sum(M, 1), B)
    # Sum across cols to get the indegrees for each block
    d_in = reshape(sum(M, 2), B)
    d = d_out + d_in
    return M, d_out, d_in, d
end

initalize_edge_counts

## Propose a new block assignment for the current node or block

### Parameters
    r : Int64
            current block assignment for the node under consideration
    neighbors_out : Array{Int64, 2}, has 2 columns.
            out neighbors for the block
    neighbors_in : Array{Int64, 2}, has 2 columns.
            in neighbors for the block
    b : Vector{Int64}
        array of block assignment for each node
    M : Array{Int64, 2}, size is (B, B)
            edge count matrix between all the blocks.
    d : Vector{Int}
            total number of edges to and from each block
    B : Int64
            total number of blocks
    agg_move : Bool
            whether the proposal is a block move

### Returns
    s : int
            proposed block assignment for the node under consideration
    k_out : int
            the out degree of the node
    k_in : int
            the in degree of the node
    k : int
            the total degree of the node

### Notes
- $d_u$: degree of block u

Randomly select a neighbor of the current node, and obtain its block assignment $u$. With probability $\frac{B}{d_u + B}$, randomly propose
a block. Otherwise, randomly selects a neighbor to block $u$ and propose its block assignment. For block (agglomerative) moves,
avoid proposing the current block.

In [67]:
function propose_new_partition(
        r::Int64, neighbors_out::Array{Int64, 2}, neighbors_in::Array{Int64, 2}, b::Vector{Int64}, M::Array{Int64, 2},
        d::Vector{Int64}, B::Int64, agg_move::Bool
    )
    neighbors = vcat(neighbors_out, neighbors_in)
    k_out = sum(neighbors_out[:, 1])
    k_in = sum(neighbors_in[:, 1])
    k = k_out + k_in
    rand_neighbor = sample(neighbors[:,0], pweights(neighbors[:,1]/k))
    u = b[rand_neighbor]
    # propose a new block randomly
    if rand() <= B/(d[u] + B)  # chance inversely prop. to block_degree
        if agg_move  # force proposal to be different from current block
            candidates = set(1:B)
            pop!(candidates, r)
            s = sample(collect(candidates))
        else
            s = rand(1:B)
        end
    else  # propose by random draw from neighbors of block partition[rand_neighbor]
        multinomial_prob = M[:, u] + M[u, :] / d[u]
        if agg_move  # force proposal to be different from current block
            multinomial_prob[r] = 0
            if sum(multinomial_prob) == 0  # the current block has no neighbors. randomly propose a different block
                candidates = set(1:B)
                pop!(candidates, r)
                s = sample(collect(candidates))
                return s, k_out, k_in, k
            else
                multinomial_prob = multinomial_prob / sum(multinomial_prob)
            end
        candidates_vec =  findn(multinomial_prob)
        s = candidates_vec[findn(rand(Multinomial(1, multinomial_prob[candidates_vec])))[1]]
        end
    end
    return s, k_out, k_in, k
end

propose_new_partition (generic function with 1 method)

## Compute the two new rows and cols of the edge count matrix under the proposal for the current node or block

### Parameters
    M : ndarray or sparse matrix (int), shape = (#blocks, #blocks)
            edge count matrix between all the blocks.
    r : int
            current block assignment for the node under consideration
    s : int
            proposed block assignment for the node under consideration
    b_out : ndarray (int)
            blocks of the out neighbors
    count_out : ndarray (int)
            edge counts to the out neighbor blocks
    b_in : ndarray (int)
            blocks of the in neighbors
    count_in : ndarray (int)
            edge counts to the in neighbor blocks
    count_self : int
            edge counts to self
    agg_move : bool
            whether the proposal is a block move
    use_sparse : bool
            whether the edge count matrix is stored as a sparse matrix

### Returns
    M_r_row : ndarray or sparse matrix (int)
            the current block row of the new edge count matrix under proposal
    M_s_row : ndarray or sparse matrix (int)
            the proposed block row of the new edge count matrix under proposal
    M_r_col : ndarray or sparse matrix (int)
            the current block col of the new edge count matrix under proposal
    M_s_col : ndarray or sparse matrix (int)
            the proposed block col of the new edge count matrix under proposal

### Notes
The updates only involve changing the entries to and from the neighboring blocks

In [76]:
function compute_new_rows_cols_interblock_edge_count_matrix(
        M::Array{Int64, 2}, r::Int64, s::Int64, b_out::Vector{Int64}, count_out::Vector{Int64}, 
        b_in::Vector{Int64}, count_in::Vector{Int64}, count_self::Int64, agg_move::Bool)
    B = size(M, 1)
    M_r_row = zeros(B)
    M_r_col = zeros(B)
    if agg_move == false
        M_r_row = M[r, :]
        M_r_col = M[:, r]
        
        M_r_col[b_out] -= count_out
        M_r_col[r] -= sum(count_in[find(x->x==r, b_in)])
        M_r_col[s] += sum(count_in[find(x->x==r, b_in)])
        
        M_r_row[b_in] -= count_in
        M_r_row[r] -= sum(count_out[find(x->x==r, b_out)])
        M_r_row[s] += sum(count_out[find(x->x==r, b_out)])
    end
    M_s_row = M[s, :]
    M_s_col = M[:, s]
    
    M_s_col[b_out] += count_out
    M_s_col[r] -= count_self
    M_s_col[s] += count_self
    M_s_col[r] -= sum(count_in[find(x->x==s, b_in)])
    M_s_col[s] += sum(count_in[find(x->x==s, b_in)])
    
    M_s_row[b_in] += count_in
    M_s_row[r] -= sum(count_out[find(x->x==s, b_out)])
    M_s_row[s] += sum(count_out[find(x->x==s, b_out)])
    M_s_row[r] -= count_self
    M_s_row[s] += count_self
    
    return M_r_row, M_s_row, M_r_col, M_s_col
end

compute_new_rows_cols_interblock_edge_count_matrix (generic function with 1 method)

## Compute the new block degrees under the proposal for the current node or block

### Parameters
    r : int
            current block assignment for the node under consideration
    s : int
            proposed block assignment for the node under consideration
    d_out : ndarray (int)
            the current out degree of each block
    d_in : ndarray (int)
            the current in degree of each block
    d : ndarray (int)
            the current total degree of each block
    k_out : int
            the out degree of the node
    k_in : int
            the in degree of the node
    k : int
            the total degree of the node

### Returns
    d_out_new : ndarray (int)
            the new out degree of each block under proposal
    d_in_new : ndarray (int)
            the new in degree of each block under proposal
    d_new : ndarray (int)
            the new total degree of each block under proposal

### Notes
The updates only involve changing the degrees of the current and proposed block

In [81]:
function compute_new_block_degrees(
        r::Int64, s::Int64, d_out::Vector{Int64}, d_in::Vector{Int64}, d::Vector{Int64}, 
        k_out::Int64, k_in::Int64, k::Int64
    )
    new_degrees = [copy(degrees) for degrees in [d_out, d_in, d]]
    for (new_d, degree) in zip(new_degrees, [k_out, k_in, k])
        new_d[r] -= degree
        new_d[s] += degree
    end
    return new_degrees
end

compute_new_block_degrees (generic function with 1 method)

## Compute change in entropy under the proposal
Reduction in entropy means the proposed block is better than the current block.

### Parameters
    r : int
            current block assignment for the node under consideration
    s : int
            proposed block assignment for the node under consideration
    M : ndarray or sparse matrix (int), shape = (#blocks, #blocks)
            edge count matrix between all the blocks.
    M_r_row : ndarray or sparse matrix (int)
            the current block row of the new edge count matrix under proposal
    M_s_row : ndarray or sparse matrix (int)
            the proposed block row of the new edge count matrix under proposal
    M_r_col : ndarray or sparse matrix (int)
            the current block col of the new edge count matrix under proposal
    M_s_col : ndarray or sparse matrix (int)
            the proposed block col of the new edge count matrix under proposal
    d_out : ndarray (int)
            the current out degree of each block
    d_in : ndarray (int)
            the current in degree of each block
    d_out_new : ndarray (int)
            the new out degree of each block under proposal
    d_in_new : ndarray (int)
            the new in degree of each block under proposal
    use_sparse : bool
            whether the edge count matrix is stored as a sparse matrix

### Returns
    delta_entropy : float
            entropy under the proposal minus the current entropy

### Notes
- $M^-$: current edge count matrix between the blocks
- $M^+$: new edge count matrix under the proposal
- $d^-_{t, \rm in}$: current in degree of block $t$
- $d^-_{t, \rm out}$: current out degree of block $t$
- $d^+_{t, \rm in}$: new in degree of block $t$ under the proposal
- $d^+_{t, \rm out}$: new out degree of block $t$ under the proposal

The difference in entropy is computed as:

$\large \Delta S = \sum_{t_1, t_2} {\left[ -M_{t_1 t_2}^+ \log\left(\frac{M_{t_1 t_2}^+}{d_{t_1, \rm out}^+ d_{t_2, \rm in}^+}\right) + M_{t_1 t_2}^- \log\left(\frac{M_{t_1 t_2}^-}{d_{t_1, \rm out}^- d_{t_2, \rm in}^-}\right)\right]}$

where the sum runs over all entries $(t_1, t_2)$ in rows and cols $r$ and $s$ of the edge count matrix

In [83]:
function compute_delta_entropy(
        r::Int64, s::Int64, M::Array{Int64, 2}, M_r_row::Vector{Int64}, M_s_row::Vector{Int64}, M_r_col::Vector{Int64},
        M_s_col::Vector{Int64}, d_out::Vector{Int64}, d_in::Vector{Int64}, d_out_new::Vector{Int64}, 
        d_in_new::Vector{Int64}
    )
    
    #TODO: Test!
    M_r_t1 = M[:, r]
    M_s_t1 = M[:, s]
    M_t2_r = M[r, :]
    M_t2_s = M[s, :]

    # remove r and s from the cols to avoid double counting
    idx = filter(x-> x∉[r,s], 1:length(d_in_new))
    
    M_r_col = M_r_col[idx]
    M_s_col = M_s_col[idx]
    
    M_t2_r = M_t2_r[idx]
    M_t2_s = M_t2_s[idx]
    
    d_out_new_ = d_out_new[idx]
    d_out_ = d_out[idx]

    # only keep non-zero entries to avoid unnecessary computation
    d_in_new_r_row = d_in_new[findn(M_r_row)]
    d_in_new_s_row = d_in_new[findn(M_s_row)]
    M_r_row = M_r_row[findn(M_r_row)]
    M_s_row = M_s_row[findn(M_s_row)]
    d_out_new_r_col = d_out_new_[findn(M_r_col)]
    d_out_new_s_col = d_out_new_[findn(M_s_col)]
    M_r_col = M_r_col[findn(M_r_col)]
    M_s_col = M_s_col[findn(M_s_col)]
    d_in_r_t1 = d_in[findn(M_r_t1)]
    d_in_s_t1 = d_in[findn(M_s_t1)]
    M_r_t1= M_r_t1[findn(M_r_t1)]
    M_s_t1 = M_s_t1[findn(M_s_t1)]
    d_out_r_col = d_out_[findn(M_t2_r)]
    d_out_s_col = d_out_[findn(M_t2_s)]
    M_t2_r = M_t2_r[findn(M_t2_r)]
    M_t2_s = M_t2_s[findn(M_t2_s)]

    # sum over the two changed rows and cols
    delta_entropy = 0
    delta_entropy -= sum(M_r_row * np.log(M_r_row / d_in_new_r_row / d_out_new[r]))
    delta_entropy -= sum(M_s_row * np.log(M_s_row / d_in_new_s_row / d_out_new[s]))
    delta_entropy -= sum(M_r_col * np.log(M_r_col / d_out_new_r_col / d_in_new[r]))
    delta_entropy -= sum(M_s_col * np.log(M_s_col / d_out_new_s_col / d_in_new[s]))
    delta_entropy += sum(M_r_t1 * np.log(M_r_t1 / d_in_r_t1 / d_out[r]))
    delta_entropy += sum(M_s_t1 * np.log(M_s_t1 / d_in_s_t1 / d_out[s]))
    delta_entropy += sum(M_t2_r * np.log(M_t2_r / d_out_r_col / d_in[r]))
    delta_entropy += sum(M_t2_s * np.log(M_t2_s / d_out_s_col / d_in[s]))
    return delta_entropy
end

compute_delta_entropy (generic function with 1 method)

## Execute the best merge (agglomerative) moves to reduce a set number of blocks

### Parameters
    delta_entropy_for_each_block : ndarray (float)
            the delta entropy for merging each block
    best_merge_for_each_block : ndarray (int)
            the best block to merge with for each block
    b : ndarray (int)
            array of block assignment for each node
    B : int
            total number of blocks in the current partition
    B_to_merge : int
            the number of blocks to merge

### Returns
    b : ndarray (int)
            array of new block assignment for each node after the merge
    B : int
            total number of blocks after the merge

In [ ]:
function carry_out_best_merges(
        delta_entropy_for_each_block::Vector{Int64}, best_merge_for_each_block::Vector{Int64}, 
        b::Vector{Int64}, B::Int64, B_to_merge::Int64
    )
    bestMerges = delta_entropy_for_each_block.argsort()
    block_map = 1:B
    num_merge = 0
    counter = 0
    while num_merge < B_to_merge:
        mergeFrom = bestMerges[counter]
        mergeTo = block_map[best_merge_for_each_block[bestMerges[counter]]]
        counter += 1
        if mergeTo != mergeFrom:
            block_map[np.where(block_map == mergeFrom)] = mergeTo
            b[np.where(b == mergeFrom)] = mergeTo
            num_merge += 1
    remaining_blocks = np.unique(b)
    mapping = -np.ones(B, dtype=int)
    mapping[remaining_blocks] = np.arange(len(remaining_blocks))
    b = mapping[b]
    B -= B_to_merge
    return b, B

In [60]:
function main(num_vertices::Int64, base_filename::String)
    #input_filename = '../../data/static/simulated_blockmodel_graph_500_nodes'
    #true_partition_available = true
    #visualize_graph = True # whether to plot the graph layout colored with intermediate partitions
    #verbose = True # whether to print updates of the partitioning
    
    # Create the graph
    g = DiGraph(num_vertices)
    # Load the first part of the graph
    load_graph!(g, base_filename, 1)
    info(g, "Loaded")

    # initialize by putting each node in its own block (N blocks)
    num_blocks = num_vertices
    partition = collect(1:num_vertices)

    # partition update parameters
    β = 3 # exploitation versus exploration (higher value favors exploitation)

    # agglomerative partition update parameters
    num_agg_proposals_per_block = 10 # number of proposals per block
    num_block_reduction_rate = 0.5 # fraction of blocks to reduce until the golden ratio bracket is established

    # nodal partition updates parameters
    max_num_nodal_itr = 100 # maximum number of iterations
    delta_entropy_threshold1 = 5e-4 # stop iterating when the change in entropy falls below this fraction of the overall entropy
                                    # lowering this threshold results in more nodal update iterations and likely better performance, but longer runtime
    delta_entropy_threshold2 = 1e-4 # threshold after the golden ratio bracket is established (typically lower to fine-tune to partition) 
    delta_entropy_moving_avg_window = 3 # width of the moving average window for the delta entropy convergence criterion

    # initialize edge counts and block degrees
    interblock_edge_count, block_degrees_out, block_degrees_in, block_degrees = \
        initialize_edge_counts(g, num_blocks, partition)

    # initialize items before iterations to find the partition with the optimal number of blocks
    optimal_B_found = false
    old_b = [[], [], []]  # partition for the high, best, and low number of blocks so far
    old_M = [[], [], []]  # edge count matrix for the high, best, and low number of blocks so far
    old_d = [[], [], []]  # block degrees for the high, best, and low number of blocks so far
    old_d_out = [[], [], []]  # out block degrees for the high, best, and low number of blocks so far
    old_d_in = [[], [], []]  # in block degrees for the high, best, and low number of blocks so far
    old_S = [Inf, Inf, Inf] # overall entropy for the high, best, and low number of blocks so far
    old_B = [[], [], []]  # number of blocks for the high, best, and low number of blocks so far
    
    
    num_blocks_to_merge = floor(Int64, num_blocks*num_block_reduction_rate)

    # begin partitioning by finding the best partition with the optimal number of blocks
    while optimal_num_blocks_found == false
        # begin agglomerative partition updates (i.e. block merging)
        println("\nMerging down blocks from $num_blocks to $(num_blocks - num_blocks_to_merge)")
        
        best_merge_for_each_block = fill(-1, num_blocks) # initialize to no merge
        delta_entropy_for_each_block = fill(Inf, num_blocks) # initialize criterion
        block_partition = 1:num_blocks
        for current_block in 1:num_blocks # evalaute agglomerative updates for each block
            for proposal_idx in 1:num_agg_proposals_per_block
                # populate edges to neighboring blocks
                out_blocks = findn(interblock_edge_count[:, current_block])
                out_blocks_counts = hcat(out_blocks, interblock_edge_count[out_blocks, current_block])
                in_blocks = findn(interblock_edge_count[current_block, :])
                in_blocks_counts = hcat(in_blocks, interblock_edge_count[current_block, in_blocks])

                # propose a new block to merge with
                proposal, num_out_neighbor_edges, num_in_neighbor_edges, num_neighbor_edges = \
                    propose_new_partition(
                        current_block, out_blocks_counts, in_blocks_counts, block_partition, 
                        interblock_edge_count, block_degrees, num_blocks, 1
                    )

                # compute the two new rows and columns of the interblock edge count matrix
                new_interblock_edge_count_current_block_row, new_interblock_edge_count_new_block_row, new_interblock_edge_count_current_block_col, new_interblock_edge_count_new_block_col = \
                    compute_new_rows_cols_interblock_edge_count_matrix(interblock_edge_count, current_block, proposal, out_blocks[:,0], out_blocks[:,1], in_blocks[:,0], in_blocks[:,1], interblock_edge_count[current_block, current_block], 1)    

                # compute new block degrees           
                block_degrees_out_new, block_degrees_in_new, block_degrees_new = \
                    compute_new_block_degrees(
                        current_block, proposal, block_degrees_out, block_degrees_in, block_degrees, 
                        num_out_neighbor_edges, num_in_neighbor_edges, num_neighbor_edges
                )

                # compute change in entropy / posterior
                delta_entropy = compute_delta_entropy(current_block, proposal, interblock_edge_count, new_interblock_edge_count_current_block_row, new_interblock_edge_count_new_block_row, new_interblock_edge_count_current_block_col, new_interblock_edge_count_new_block_col, block_degrees_out, block_degrees_in, block_degrees_out_new, block_degrees_in_new, use_sparse_matrix)
                if delta_entropy < delta_entropy_for_each_block[current_block]: # a better block candidate was found
                    best_merge_for_each_block[current_block] = proposal
                    delta_entropy_for_each_block[current_block] = delta_entropy

        # carry out the best merges
        partition, num_blocks = carry_out_best_merges(delta_entropy_for_each_block, best_merge_for_each_block, partition, num_blocks, num_blocks_to_merge)

        # re-initialize edge counts and block degrees
        interblock_edge_count, block_degrees_out, block_degrees_in, block_degrees = initialize_edge_counts(out_neighbors, num_blocks, partition, use_sparse_matrix)

        # perform nodal partition updates
        if verbose:
            print("Beginning nodal updates")
        total_num_nodal_moves = 0            
        itr_delta_entropy = np.zeros(max_num_nodal_itr)

        # compute the global entropy for MCMC convergence criterion
        overall_entropy = compute_overall_entropy(interblock_edge_count, block_degrees_out, block_degrees_in, num_blocks, N, E, use_sparse_matrix)

        for itr in range(max_num_nodal_itr):
            num_nodal_moves = 0;
            itr_delta_entropy[itr] = 0

            for current_node in range(N):
                current_block = partition[current_node] 
                # propose a new block for this node
                proposal, num_out_neighbor_edges, num_in_neighbor_edges, num_neighbor_edges = propose_new_partition(current_block, out_neighbors[current_node], in_neighbors[current_node], partition, interblock_edge_count, block_degrees, num_blocks, 0, use_sparse_matrix)

                # determine whether to accept or reject the proposal
                if (proposal != current_block):
                    # compute block counts of in and out neighbors
                    blocks_out, inverse_idx_out = np.unique(partition[out_neighbors[current_node][:,0]], return_inverse = True)
                    count_out = np.bincount(inverse_idx_out, weights=out_neighbors[current_node][:,1]).astype(int)
                    blocks_in, inverse_idx_in = np.unique(partition[in_neighbors[current_node][:,0]], return_inverse = True)
                    count_in = np.bincount(inverse_idx_in, weights=in_neighbors[current_node][:,1]).astype(int)

                    # compute the two new rows and columns of the interblock edge count matrix
                    self_edge_weight = np.sum(out_neighbors[current_node][np.where(out_neighbors[current_node][:,0]==current_node),1]) # check if this node has a self edge
                    new_interblock_edge_count_current_block_row, new_interblock_edge_count_new_block_row, new_interblock_edge_count_current_block_col, new_interblock_edge_count_new_block_col = \
                        compute_new_rows_cols_interblock_edge_count_matrix(interblock_edge_count, current_block, proposal, blocks_out, count_out, blocks_in, count_in, self_edge_weight, 0, use_sparse_matrix)

                    # compute new block degrees           
                    block_degrees_out_new, block_degrees_in_new, block_degrees_new = compute_new_block_degrees(current_block, proposal, block_degrees_out, block_degrees_in, block_degrees, num_out_neighbor_edges, num_in_neighbor_edges, num_neighbor_edges)

                    # compute the Hastings correction
                    Hastings_correction = compute_Hastings_correction(blocks_out, count_out, blocks_in, count_in, proposal, interblock_edge_count, new_interblock_edge_count_current_block_row, new_interblock_edge_count_current_block_col, num_blocks, block_degrees, block_degrees_new, use_sparse_matrix)

                    # compute change in entropy / posterior
                    delta_entropy = compute_delta_entropy(current_block, proposal, interblock_edge_count, new_interblock_edge_count_current_block_row, new_interblock_edge_count_new_block_row, new_interblock_edge_count_current_block_col, new_interblock_edge_count_new_block_col, block_degrees_out, block_degrees_in, block_degrees_out_new, block_degrees_in_new, use_sparse_matrix)

                    # compute probability of acceptance
                    p_accept = np.min([np.exp(-beta*delta_entropy)*Hastings_correction, 1])

                    # if accept the proposal, update the partition, inter_block_edge_count, and block degrees
                    if (np.random.uniform() <= p_accept):
                        total_num_nodal_moves += 1
                        num_nodal_moves += 1
                        itr_delta_entropy[itr] += delta_entropy
                        partition, interblock_edge_count, block_degrees_out, block_degrees_in, block_degrees = update_partition(partition, current_node, current_block, proposal, interblock_edge_count, new_interblock_edge_count_current_block_row, new_interblock_edge_count_new_block_row, new_interblock_edge_count_current_block_col, new_interblock_edge_count_new_block_col, block_degrees_out_new, block_degrees_in_new, block_degrees_new, use_sparse_matrix)
            if verbose:
                print("Itr: {}, number of nodal moves: {}, delta S: {:0.5f}".format(itr, num_nodal_moves, itr_delta_entropy[itr]/float(overall_entropy)))
            if itr>=(delta_entropy_moving_avg_window-1): # exit MCMC if the recent change in entropy falls below a small fraction of the overall entropy
                if not(np.all(np.isfinite(old_overall_entropy))): # golden ratio bracket not yet established 
                    if (-np.mean(itr_delta_entropy[(itr-delta_entropy_moving_avg_window+1):itr]) < (delta_entropy_threshold1*overall_entropy)):
                        break
                else: # golden ratio bracket is established. Fine-tuning partition.
                    if (-np.mean(itr_delta_entropy[(itr-delta_entropy_moving_avg_window+1):itr]) < (delta_entropy_threshold2*overall_entropy)):
                        break

        # compute the global entropy for determining the optimal number of blocks
        overall_entropy = compute_overall_entropy(interblock_edge_count, block_degrees_out, block_degrees_in, num_blocks, N, E, use_sparse_matrix)

        if verbose:
            print("Total number of nodal moves: {}, overall_entropy: {:0.2f}".format(total_num_nodal_moves, overall_entropy))
        if visualize_graph & use_graph_tool_options:
            graph_object = plot_graph_with_partition(out_neighbors, partition, graph_object)

        # check whether the partition with optimal number of block has been found; if not, determine and prepare for the next number of blocks to try
        partition, interblock_edge_count, block_degrees, block_degrees_out, block_degrees_in, num_blocks, num_blocks_to_merge, old_partition, old_interblock_edge_count, old_block_degrees, old_block_degrees_out, old_block_degrees_in, old_overall_entropy, old_num_blocks, optimal_num_blocks_found = \
            prepare_for_partition_on_next_num_blocks(overall_entropy, partition, interblock_edge_count, block_degrees, block_degrees_out, block_degrees_in, num_blocks, old_partition, old_interblock_edge_count, old_block_degrees, old_block_degrees_out, old_block_degrees_in, old_overall_entropy, old_num_blocks, num_block_reduction_rate)

        if verbose:
            print('Overall entropy: {}'.format(old_overall_entropy))
            print('Number of blocks: {}'.format(old_num_blocks))
            if optimal_num_blocks_found:
                print('\nOptimal partition found with {} blocks'.format(num_blocks))
    if use_timeit:
        t1 = timeit.default_timer()
        print('\nGraph partition took {} seconds'.format(t1-t0))

    # evaluate output partition against the true partition
    evaluate_partition(true_partition, partition)
end

LoadError: [91msyntax: line break in ":" expression[39m

In [72]:
no

LoadError: [91mUndefVarError: no not defined[39m

In [73]:
np

LoadError: [91mUndefVarError: np not defined[39m